# Xearch 
## Exoplanetary Laboratory
### By Jorge I. Zuluaga

In [175]:
# External modules
import spiceypy as spy
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as c
import pandas as pd
from ipywidgets import interact, widgets, interactive, fixed, interact_manual
# Read planetary systems
exec(open('systems-full.py').read())
# Constants
deg = np.pi/180
rad = 1/deg
Msun = c.M_sun.value
Mearth = c.M_earth.value
Mjup = c.M_jup.value
AU = c.au.value 
G = c.G.value
Rsun = c.R_sun.value
Rearth = c.R_earth.value
Rjup = c.R_jup.value
days = 86400
years = 365.25*days

In [180]:
system = '51 Peg b'
system = 'GJ3512 b'
system = 'Proxima Centauri b'

In [182]:
# Read data
df = pd.read_csv(f"signals/{system.replace(' ','_')}.csv")

# Read extradata
UM = df.UM.iloc[0]
UL = df.UL.iloc[0]
UT = df.UT.iloc[0]
UV = UL/UT
stellar_mass = df.stellar_mass.iloc[0]
planet_mass = df.planet_mass.iloc[0]*UM/Mjup

# Read statistics
tmin = df.tobs.min()/days
tmax = df.tobs.max()/days
vmin = df.vrads.min()
vmax = df.vrads.max()
vrange = vmax - vmin
duration = tmax - tmin

# Estimate planetary mass
K = (df.vrads.max()-df.vrads.min())/2

In [199]:
#@title
@interact(
    tini=widgets.FloatSlider(
        description='Initial time:',
        min=tmin,max=tmax,step=duration/500,value=tmin,
        continuous_update=False
    ),
    gperiod=widgets.FloatSlider(
        description='Period: ',
        min=0,max=tmax,step=duration/500,value=duration/3,
        continuous_update=False
    ),
    m2=widgets.FloatSlider(
        description='Planet mass: ',
        min=planet_mass/100,max=10*planet_mass,step=planet_mass/500,value=planet_mass/3,
        readout_format='.5f',
        continuous_update=False
    ),
    e=widgets.FloatSlider(
        description='Eccentricity: ',
        min=0,max=1,step=1/100,value=0,
        continuous_update=False
    ),
    I=widgets.FloatSlider(
        description='Inclination: ',
        min=0,max=90,step=90/500,value=0,
        continuous_update=False
    ),
)
def plot_data(tini,gperiod,m2,e,I,signal=False):

    # Plot Data
    fig,axs = plt.subplots(1,1,figsize=(10,5))
    ax = axs
    ax.errorbar(df.tobs/days,df.vrads,df.dvrads,color='k',fmt='o',capsize=2)
    ax.set_xlabel(f"Time [days]")
    ax.set_ylabel(f"Radial Velocity [m/s]")
    
    # Plot period
    tmarks = np.arange(tini,tmax,gperiod)
    for tm in tmarks:
        ax.axvline(tm,ls='--',color='r')
    
    m2 = m2*Mjup/UM
    if signal:
        # Plot guessed signal
        ts = np.linspace(tmin,tmax,1000)

        # System parameters
        m1 = stellar_mass
        mu = m1 + m2
        T = (gperiod*days/UT)
        a = (mu/(2*np.pi/T)**2)**(1/3)
        e = e
        I = I*deg
        # Additional parameters
        q = a*(1 - e)
        W = 90*deg
        w = 0*deg
        M = -120*deg

        Nt = 1000
        ts = np.linspace(tmin*days/UT,tmax*days/UT,Nt)
        Xs = np.zeros((Nt,6))
        elements = [q, e, I, W, w, M, 0, mu]
        for i,t in enumerate(ts):
            Xs[i] = m2/mu*spy.conics(elements, t)    
        
        vrad_teos = Xs[:,3]*UV
        ax.plot(ts*UT/days,vrad_teos,'b-',alpha=0.3,zorder=-100)
        ax.set_xlabel(f"Time [days]")
        ax.set_ylabel(f"Radial Velocity [m/s]")
        ax.axhline(vrad_teos.max(),color='k',lw=5,alpha=0.2)
        ax.axhline(vrad_teos.min(),color='k',lw=5,alpha=0.2)


    # Decoration
    ax.grid()
    ax.set_xlim(tmin-duration/100,tmax)

    ax.set_title(rf"{system}, P = {gperiod:.2f} days, $m_2$ = {m2*UM/Mjup:.2f} $M_{{\rm jup}}$, e = {e:.2f}, I = {I*rad:.1f} deg",fontsize=13)
    
    fig.show()

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Initial time:', max=111.748…